##### ЗАДАЧИ ДЛЯ ПРАКТИКИ ООП

**1. БАЗОВЫЕ ФИНАНСОВЫЕ ИНСТРУМЕНТЫ:**
- Реализуйте класс `EuropeanPutOption`, наследующий от `Option`
- Реализуйте класс `AmericanCallOption` с возможностью досрочного исполнения
- Добавьте валидацию даты истечения в базовый класс `Option`
- Создайте класс `Stock` (акция) как наследник `AbstractAsset`

In [ ]:
# TODO
# 1) добавить валидацию в Option
# 2) реализовать AmericanOption

In [ ]:
from abc import ABC, abstractmethod
import math
from datetime import datetime
import time
from scipy.stats import norm

###         Inheritance strategy
# AbstractAsset -> Derivative -> Option -> European
# AbstractAsset -> Stock

class AbstractAsset(ABC):
    def __init__(self, name: str, price: float):
        self.name = name
        self.price = price
    def __repr__(self) -> str:
        return f'Stock: {self.name}, \nPrice: {self.price}'


class Stock:...

class Stock(AbstractAsset):

    __registry: dict[str, Stock] = dict()

    @classmethod
    def get_from_registry(cls, name: str) -> Stock:
        if (stock := cls.__registry.get(name)) is None: 
            raise KeyError(f"Stock {name} doesn't exist")
        return stock

    def __new__(cls, *args, **kwargs):
        instance = super().__new__(cls)
        name = kwargs.get('name')
        cls.__registry[name] = instance
        return instance
    
    
class Derivative(AbstractAsset):
    FORMAT_CODE = "%Y-%m-%d"
    def __init__(self, *args, expiration_date: str, underlying_name: str, **kwargs):
        super().__init__(*args, **kwargs)
        self.expiration_date = expiration_date
        self.__underlying: Stock = Stock.get_from_registry(underlying_name)
    
    @property
    def underlying(self) -> Stock:
        return self.__underlying

class Option(Derivative):
    def __init__(self, *args, strike_price: float, risk_free_rate: float, volatility: float, **kwargs):
        super().__init__(*args, **kwargs)
        self.strike_price = strike_price
        self.risk_free_rate = risk_free_rate
        self.volatility = volatility
        self.__payoff = None

    # TODO: валидация в Option == проверить, что введенная дата не раньше даты из os
            
    @abstractmethod
    def exercise(self):...


class EuropeanCallOption(Option):
    @property
    def payoff(self) -> float:
        T: float = datetime.strptime(self.expiration_date, self.FORMAT_CODE).timestamp() - time.time()
        T /= 60*60*24*365
        d1 = (math.log(self.underlying.price / self.strike_price) +\
             (self.risk_free_rate + self.volatility**2 / 2) * T) /\
             (self.volatility * math.sqrt(T))
        d2 = d1 - (self.volatility * math.sqrt(T))
        c = self.underlying.price * norm.cdf(d1) - self.strike_price * \
            math.exp(-self.risk_free_rate) * norm.cdf(d2)
        return c
    def exercise(self):
        # TODO:
        # EUROPEAN OPTION CASE: if data_exp != today => exception
        # AMERICAN OPTION CASE: no need
        return max(self.underlying.price - self.strike_price, 0)

# TODO: implement american option
class AmericanOption(Option):...


In [4]:
vtbr = Stock(name = 'VTBR', price = 70)

eur_call_option = EuropeanCallOption(
    name = 'Default Name', 
    strike_price = 70,
    risk_free_rate = 0.18,
    volatility = 0.3,
    expiration_date = '2025-12-31',
    price = 7,
    underlying_name = 'VTBR'
)

In [6]:
sber: Stock = Stock(name = 'SBER', price = 300)
# Stock.get_registry('SBER')

In [11]:
eur_call_option.payoff

np.float64(10.761856373105637)

**2. РАСШИРЕННЫЕ ОПЦИОНЫ:**
- Реализуйте класс `BinaryOption` (бинарный опцион)
- Создайте класс `BarrierOption` (барьерный опцион)
- Добавьте метод расчета Greeks (дельта, гамма, тета, вега) для опционов

In [ ]:
# TODO
# 1) реализовать BinaryOption(fixed_payoff)
# 2) реализовать BarrierOption(P>0 if S_t in [B_L, B_U])
# 3) Greeks сделать без рассчетов (pass), "этот метод существует"

**3. ПОРТФЕЛЬ И УПРАВЛЕНИЕ АКТИВАМИ:**
- Реализуйте метод `__repr__` для класса `Portfolio`
- Добавьте метод `__getitem__` для доступа к активам по индексу
- Реализуйте метод `__contains__` для проверки наличия актива в портфеле
- Добавьте метод `remove_asset` для удаления актива из портфеля
- Создайте метод `calculate_portfolio_risk` для расчета риска портфеля

In [ ]:
# TODO
# 1) реализовать Portfolio (все хранится в словаре)
# 2) calculate_portfolio_risk - без реализации (pass)

**5. ДОПОЛНИТЕЛЬНЫЕ DUNDER МЕТОДЫ:**
- Добавьте `__eq__`, `__lt__`, `__le__` для сравнения портфелей
- Реализуйте `__mul__` для умножения портфеля на число (масштабирование)
- Добавьте `__str__` для красивого вывода портфеля
- Реализуйте `__bool__` для проверки, является ли портфель пустым

**6. ПРОДВИНУТЫЕ ЗАДАЧИ:**
- Создайте декоратор `@validate_option_params` для валидации параметров опционов
- Реализуйте класс `PortfolioManager` с методами для управления несколькими портфелями
- Добавьте поддержку сериализации/десериализации портфелей (pickle)
- Создайте контекстный менеджер для временного изменения параметров портфеля

In [ ]:
# TODO
# 1) реализовать PortfolioManager (принимает на вход список портфелей, методы убрать и добавить портфель)
# 2) добавить сериализацию/десериализацию через pickle
# 3) контекстный менеджер